# Conditional GAN

##### GAN vs cGAN
![](https://nooverfit.com/wp/wp-content/uploads/2017/10/Screenshot-from-2017-10-07-120039.png)

## Preparation

### Load the Library

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib as pyplot

from datetime import datetime

tf.__version__

'2.7.4'

### Prepare Training Data

In [2]:
# Labels are needed to add conditions.
(train_x, train_y), (_, _) = tf.keras.datasets.mnist.load_data()

# Preprocessing
# Normalization: (0~255) -> (-1~1), Expand dims: 3(None, ?, ?) -> 4(None, ?, ?, 1)
train_x = train_x / 127.5 - 1
train_x = tf.expand_dims(train_x, axis=-1)

2022-10-07 17:09:41.187998: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 17:09:43.028036: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22836 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:5e:00.0, compute capability: 7.5
2022-10-07 17:09:43.030992: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22230 MB memory:  -> device: 1, name: NVIDIA TITAN RTX, pci bus id: 0000:af:00.0, compute capability: 7.5


### Declare a Global Variables

In [3]:
INPUT_SHAPE = train_x.shape 

DATA_SHAPE = INPUT_SHAPE[1:]
DATA_ROWS = INPUT_SHAPE[1]
DATA_COLS = INPUT_SHAPE[2]
DATA_CHANNELS = INPUT_SHAPE[3]
DATA_SIZE = np.prod(DATA_SHAPE) # DATA_SHAPE[0]*[1]*[2]

LATENT_Z_DIMS = 100
NUM_CLASSES = 10

EPOCHS = 10000
BATCH_SIZE = 256
NUM_DISPLAY_LOG = 20
EPOCH_DISPLAY_LOG = EPOCHS // NUM_DISPLAY_LOG

REAL_LABELS, FAKE_LABELS = np.ones((BATCH_SIZE, 1)), np.zeros((BATCH_SIZE, 1))

## Define Models

### Generative Model

In [7]:
def build_generator():
    model = tf.keras.models.Sequential([
        # LATENT_Z_DIMS -> units(=7*7*256), 7 -> 28
        tf.keras.layers.Dense(units=7*7*256, input_dim=LATENT_Z_DIMS),
        tf.keras.layers.Reshape((7,7,256)),
        
        # (7,7,256) -> (14,14,128): strides = (2,2)
        tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(3,3), strides=(2,2), padding="same"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.01),
        
        # (14,14,128) -> (28,28,1)
        tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=(3,3), strides=(2,2), padding="same"),
        tf.keras.layers.Activation("tanh")
    ])
    return model

### Generative Model with Conditions

In [12]:
def build_cgan_generator():
    # Noise vector: (None, LATENT_Z_DIMS)
    noise_vec = tf.keras.layers.Input(shape=(LATENT_Z_DIMS))
    # Conditional labels: Int 0~9(None, 1)
    condition_labels = tf.keras.layers.Input(shape=(1,), dtype="int32")
    # (None, 1) -> (None, 1, LATENT_Z_DIMS): value is int 0 to 9
    embedding_labels = tf.keras.layers.Embedding(
        input_dim=NUM_CLASSES, output_dim=LATENT_Z_DIMS, input_length=1
    )(condition_labels)
    # (None, 1, LATENT_Z_DIMS) -> (None, LATENT_Z_DIMS)
    embedding_labels = tf.keras.layers.Flatten()(embedding_labels)
    # Noise vector with conditions
    latent_data = tf.keras.layers.Multiply()([noise_vec, embedding_labels])
    generator = build_generator()
    gen_images_with_conditions = generator(latent_data)
    
    return tf.keras.models.Model(input=[noise_vec, condition_labels], 
                                 output=gen_images_with_conditions)

### Discriminative Model

In [13]:
def build_discriminator():
    model = tf.keras.models.Sequential([
        # Receive data(28,28,2) = condition information added in the channel
        # to (14,14,64)
        # Convolution #1
        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), padding="same",
                               input_shape=(DATA_ROWS, DATA_COLS, DATA_CHANNELS+1)),
        tf.keras.layers.LeakyReLU(alpha=0.01),
        
        # (14,14,64) -> (7,7,128)
        # Convolution #2
        tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(2,2), padding="same"),
        tf.keras.layers.LeakyReLU(alpha=0.01),
        
        # (7,7,128) -> (3,3,256)
        # Convolution #3
        tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(2,2), padding="same"),
        tf.keras.layers.LeakyReLU(alpha=0.01),
        
        tf.keras.layers.Flatten(),
        
        # (3*3*256) -> (1)
        # Full-Connected Layer
        tf.keras.layers.Dense(units=1, activation="sigmoid")
    ])
    return model